In [13]:
import numpy as np
import pandas
import matplotlib.pyplot as plt

from keras import layers, optimizers, models
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from IPython.display import Image, SVG
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [78]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.mode.chained_assignment = None

from sklearn.svm import SVR
# from skrvm import RVR
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import datetime as dt
from sklearn.multioutput import MultiOutputRegressor
from matplotlib import pyplot as plt

In [87]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.mode.chained_assignment = None

import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

In [3]:
train_data = pandas.read_csv("adult.data", encoding="UTF-8", header=None)
test_data = pandas.read_csv("adult.test", encoding="UTF-8", header=None, skiprows=1) # skip 第一行

In [4]:
test_data[14] = test_data[14].str.replace('.', '') # 將 test_data class label 最後多餘的點去掉
train_data = train_data.replace(' ?', np.nan) # 將 missing value 改成 NaN
test_data = test_data.replace(' ?', np.nan) # 將 missing value 改成 NaN

# 
# if instances with unknown values are removed (train=30162, test=15060)
# 
train_data = train_data.dropna() # 去掉 missing value 的那筆資料，剩下 30162 筆
test_data = test_data.dropna() # 去掉 missing value 的那筆資料，剩下 15060 筆

In [5]:
cols = train_data.columns.tolist() # 把train_data的columns轉換成list
cols.insert(14, cols.pop(cols.index(12))) # 調換 12 跟 14 列

cols = test_data.columns.tolist() # 把test_data的columns轉換成list
cols.insert(14,cols.pop(cols.index(12))) # 調換 12 跟 14 列
cols

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 12]

In [6]:
train_data = train_data.loc[:,cols] # set剛剛重新定位的columns
# train_data
test_data = test_data.loc[:,cols] # set剛剛重新定義的columns
# test_data
train_data.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14] # 對train_data的cloumns重新命名，讓它能夠恢復名稱順序
# train_data
test_data.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14] # 對test_data的cloumns重新命名，讓它能夠恢復名稱順序
# test_data

In [7]:
#使用 Label Encoding 把字串以數字取代
def transform_label(full_data):
    transform_needed = [False,
           True,
           False,
           True,
           False,
           True,
           True,
           True,
           True,
           True,
           False,
           False,
           True,
           True,
           False]


    result = np.zeros(shape=(full_data.shape[0], full_data.shape[1]), dtype=np.float32)

    for i in range(len(transform_needed)):
        if transform_needed[i]:
            tmp_data = full_data.iloc[:, i].tolist()
            encoder = LabelEncoder()
            encoder.fit(tmp_data)
            result[:, i] = encoder.transform(tmp_data)
        else:
            result[:, i] = full_data.iloc[:, i].tolist()
            
    return result

In [70]:
# 對 training data 和 testing data 進行 label encoding 編碼
transformed_train_data = transform_label(train_data)
transformed_test_data = transform_label(test_data)

# Feature scaling : 將資料正規化
scaler = MinMaxScaler().fit(transformed_train_data)

transformed_train_data = scaler.transform(transformed_train_data)
transformed_test_data = scaler.transform(transformed_test_data)

# 拆分成輸入資料和答案
x_train = transformed_train_data[:, :13]
y_train = transformed_train_data[:, 14]

x_test = transformed_test_data[:, :13]
y_test = transformed_test_data[:, 14]

In [71]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[0.30136985 0.8333334  0.04333771 ... 0.02174022 0.         0.95      ]
 [0.4520548  0.6666667  0.04727738 ... 0.         0.         0.95      ]
 [0.2876712  0.33333334 0.13724391 ... 0.         0.         0.95      ]
 ...
 [0.56164384 0.33333334 0.09391367 ... 0.         0.         0.95      ]
 [0.06849313 0.33333334 0.1276201  ... 0.         0.         0.95      ]
 [0.47945207 0.5        0.18638337 ... 0.1502415  0.         0.95      ]]
[0.39795917 0.12244897 0.39795917 ... 0.39795917 0.19387755 0.39795917]
[[0.10958904 0.33333334 0.1448282  ... 0.         0.         0.925     ]
 [0.2876712  0.33333334 0.05169837 ... 0.         0.         0.925     ]
 [0.15068492 0.16666667 0.21971181 ... 0.         0.         0.925     ]
 ...
 [0.2876712  0.33333334 0.24556744 ... 0.         0.         0.925     ]
 [0.36986297 0.33333334 0.04767168 ... 0.05455054 0.         0.925     ]
 [0.24657533 0.5        0.11447065 ... 0.         0.         0.925     ]]
[0.39795917 0.5        0.39795917 ... 0.

In [58]:
# 建構 model
model = models.Sequential()
model.add(layers.Dense(64, input_shape=(x_train.shape[1],), activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dropout(0.15))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, kernel_initializer="uniform",activation='linear'))


model.compile(loss='mean_squared_error', optimizer='adam')

# 開始訓練 model
history = model.fit(x_train, y_train, validation_split=0.33, epochs=100, batch_size=128)

Train on 20208 samples, validate on 9954 samples
Epoch 1/100
20208/20208 [==============================] - 1s 42us/step - loss: 0.0344 - val_loss: 0.0147
Epoch 2/100
20208/20208 [==============================] - 1s 33us/step - loss: 0.0142 - val_loss: 0.0148
Epoch 3/100
20208/20208 [==============================] - 1s 34us/step - loss: 0.0133 - val_loss: 0.0164
Epoch 4/100
20208/20208 [==============================] - 1s 33us/step - loss: 0.0129 - val_loss: 0.0132
Epoch 5/100
20208/20208 [==============================] - 1s 33us/step - loss: 0.0126 - val_loss: 0.0145
Epoch 6/100
20208/20208 [==============================] - 1s 32us/step - loss: 0.0125 - val_loss: 0.0128
Epoch 7/100
20208/20208 [==============================] - 1s 33us/step - loss: 0.0123 - val_loss: 0.0135
Epoch 8/100
20208/20208 [==============================] - 1s 32us/step - loss: 0.0122 - val_loss: 0.0125
Epoch 9/100
20208/20208 [==============================] - 1s 33us/step - loss: 0.0121 - val_loss: 0.01

In [61]:
# 顯示模型摘要與結構
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 64)                896       
_________________________________________________________________
dense_67 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_68 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 17        
Total params: 4,577
Trainable params: 4,577
Non-trainable params: 0
___________________________________________________

# ANN

In [74]:
# 評估指標

y_test = test_data[14].to_numpy().reshape(len(test_data[14].to_numpy()), 1) # 取得答案
print(y_test.shape)
scaler = MinMaxScaler().fit(y_test) # 將答案的值進行正規化

yhat = model.predict(x_test)
print(yhat.shape)
preds = scaler.inverse_transform(yhat)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
mape = np.mean(np.abs((preds - y_test) / y_test)) * 100
print("MAPE: %f" % (mape))

# 畫圖
# plt.plot(history.history['root_mean_squared_error'], label='root_mean_squared_error')
# plt.plot(history.history['loss'], label='loss')
# plt.plot(history.history['val_loss'], label='val_loss')
# plt.legend()
# plt.show()

(15060, 1)
(15060, 1)
RMSE: 10.766541
MAPE: 25.803733


# SVR

In [75]:
# 對 training data 和 testing data 進行 label encoding 編碼
transformed_train_data = transform_label(train_data)
transformed_test_data = transform_label(test_data)

# Feature scaling : 將資料正規化
scaler = MinMaxScaler().fit(transformed_train_data)

transformed_train_data = scaler.transform(transformed_train_data)
transformed_test_data = scaler.transform(transformed_test_data)

# 拆分成輸入資料和答案
x_train = transformed_train_data[:, :13]
y_train = transformed_train_data[:, 14]

x_test = transformed_test_data[:, :13]
y_test = transformed_test_data[:, 14]

In [79]:
svr = SVR(gamma='scale',epsilon=0.001, )
# svr = SVR(kernel='rbf', gamma=scale, epsilon=0.001)
# RMSE: 10.966594
# MAPE: 26.509665

svr.fit(x_train,y_train)

SVR(epsilon=0.001)

In [80]:
# 評估指標

y_test = test_data[14].to_numpy().reshape(len(test_data[14].to_numpy()), 1) # 取得答案
print(y_test.shape)
scaler = MinMaxScaler().fit(y_test) # 將答案的值進行正規化

x_test = np.array(x_test)
yhat = svr.predict(x_test)
print(yhat.shape)
preds = scaler.inverse_transform(yhat.reshape(-1, 1))
print(preds.shape)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
mape = np.mean(np.abs((preds - y_test) / y_test)) * 100
print("MAPE: %f" % (mape))

# 畫圖
# plt.plot(history.history['root_mean_squared_error'], label='root_mean_squared_error')
# plt.plot(history.history['loss'], label='loss')
# plt.plot(history.history['val_loss'], label='val_loss')
# plt.legend()
# plt.show()

(15060, 1)
(15060,)
(15060, 1)
RMSE: 10.966594
MAPE: 26.509665


# RF

In [84]:
# 對 training data 和 testing data 進行 label encoding 編碼
transformed_train_data = transform_label(train_data)
transformed_test_data = transform_label(test_data)

# Feature scaling : 將資料正規化
scaler = MinMaxScaler().fit(transformed_train_data)

transformed_train_data = scaler.transform(transformed_train_data)
transformed_test_data = scaler.transform(transformed_test_data)

# 拆分成輸入資料和答案
x_train = transformed_train_data[:, :13]
y_train = transformed_train_data[:, 14]

x_test = transformed_test_data[:, :13]
y_test = transformed_test_data[:, 14]

In [85]:
RF = RandomForestRegressor(max_depth=10, random_state=0,n_estimators = 1450)
RF.fit(x_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=1450, random_state=0)

In [86]:
# 評估指標

y_test = test_data[14].to_numpy().reshape(len(test_data[14].to_numpy()), 1) # 取得答案
print(y_test.shape)
scaler = MinMaxScaler().fit(y_test) # 將答案的值進行正規化

x_test = np.array(x_test)
yhat = RF.predict(x_test)
print(yhat.shape)
preds = scaler.inverse_transform(yhat.reshape(-1, 1))
print(preds.shape)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
mape = np.mean(np.abs((preds - y_test) / y_test)) * 100
print("MAPE: %f" % (mape))

# 畫圖
# plt.plot(history.history['root_mean_squared_error'], label='root_mean_squared_error')
# plt.plot(history.history['loss'], label='loss')
# plt.plot(history.history['val_loss'], label='val_loss')
# plt.legend()
# plt.show()

(15060, 1)
(15060,)
(15060, 1)
RMSE: 10.466266
MAPE: 26.201979


# XGBoost

In [89]:
# 對 training data 和 testing data 進行 label encoding 編碼
transformed_train_data = transform_label(train_data)
transformed_test_data = transform_label(test_data)

# Feature scaling : 將資料正規化
scaler = MinMaxScaler().fit(transformed_train_data)

transformed_train_data = scaler.transform(transformed_train_data)
transformed_test_data = scaler.transform(transformed_test_data)

# 拆分成輸入資料和答案
x_train = transformed_train_data[:, :13]
y_train = transformed_train_data[:, 14]

x_test = transformed_test_data[:, :13]
y_test = transformed_test_data[:, 14]

In [90]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', booster = 'gbtree',
                              reg_lambda = 0.0011, learning_rate = 0.1, n_estimators = 105)

xg_reg.fit(x_train,y_train)

C:\Users\IDSL\anaconda3\lib\site-packages\xgboost\core.py:444: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=105, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=0.0011, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [91]:
# 評估指標

y_test = test_data[14].to_numpy().reshape(len(test_data[14].to_numpy()), 1) # 取得答案
# print(y_test.shape)
scaler = MinMaxScaler().fit(y_test) # 將答案的值進行正規化

x_test = np.array(x_test)
yhat = xg_reg.predict(x_test)
# print(yhat.shape)
preds = scaler.inverse_transform(yhat.reshape(-1, 1))
# print(preds.shape)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
mape = np.mean(np.abs((preds - y_test) / y_test)) * 100
print("MAPE: %f" % (mape))

# 畫圖
# plt.plot(history.history['root_mean_squared_error'], label='root_mean_squared_error')
# plt.plot(history.history['loss'], label='loss')
# plt.plot(history.history['val_loss'], label='val_loss')
# plt.legend()
# plt.show()

RMSE: 10.389408
MAPE: 25.760196
